# Heat Equation - FEniCS - Manufactured Solution

### Proposed Heat Equation

\begin{equation}
    \begin{cases}
    u_{t} = u_{xx} + u_{yy}, \quad \quad (x,y) \in \Omega \\
    u' = 0, \quad\quad\quad\quad\quad\quad (x,y) \in \partial \Omega\\
    \end{cases}
\end{equation}
#### Initial Conditions
\begin{equation}
        U(x,y,0) =  f(x,y) = \cos{(\pi x)}\cos{(\pi y)}
\end{equation}

#### Exact Solution: 
\begin{equation}
        U(x,y,t) = \cos{(\pi x)}\cos{(\pi y)} e^{-2\pi^{2}t}
\end{equation}

In [16]:
from fenics import *
import numpy as np


T = 0.0005           # final time
num_steps = 5     # number of time steps
dt = T / num_steps # time step size


# Create mesh and define function space
nx = ny = 250
mesh = UnitSquareMesh(nx, ny)
V = FunctionSpace(mesh, "Lagrange", 3)

# Define boundary condition, t value can be modified later
u_D = Expression('cos(x[0]*pi)*cos(x[1]*pi)*exp(-2*t*pow(pi,2))',
                 degree=4, t=0, pi=np.pi)



# Define initial value
u_n = interpolate(u_D, V)
#u_n = project(u_D, V)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(0)

F = u*v*dx + dt*dot(grad(u), grad(v))*dx - (u_n + dt*f)*v*dx
a, L = lhs(F), rhs(F)
vtkfile = File('heat_closed_sol/solution.pvd')

# Time-stepping
u = Function(V)
t = 0
for n in range(num_steps):

    # Update current time
    t += dt
    u_D.t = t

    # Compute solution
    solve(a == L, u)

    # Compute error using analytical solution
    u_e = interpolate(u_D, V)
    error_max = np.abs(np.array(u_e.vector()) - np.array(u.vector())).max()
    error_mean = np.abs(np.array(u_e.vector()) - np.array(u.vector())).mean()

    print('t = %.5f: max error = %.10g' % (t, error_max))
    print('t = %.5f: mean error = %.10g' % (t, error_mean))


    # Save reesults
    
    vtkfile << (u, t)
    
    # Update previous solution
    u_n.assign(u)

# Hold plot


t = 0.00010: max error = 1.941820098e-06
t = 0.00010: mean error = 7.881744488e-07
t = 0.00020: max error = 3.875936167e-06
t = 0.00020: mean error = 1.57323874e-06
t = 0.00030: max error = 5.802420252e-06
t = 0.00030: mean error = 2.35520524e-06
t = 0.00040: max error = 7.721297114e-06
t = 0.00040: mean error = 3.13408311e-06
t = 0.00050: max error = 9.632602089e-06
t = 0.00050: mean error = 3.90988149e-06
